In [1]:
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument("--max_iters", type=int, default=500, help="number of epochs of training")
parser.add_argument("--batch_size", type=int, default=128, help="size of the batches")
parser.add_argument("--lr", type=float, default=0.002, help="adam: learning rate")
parser.add_argument("--struct", type=list, default=[12,9], help=" dimension of autoencoder network")
parser.add_argument("--dis_struct", type=list, default=[3,1], help=" dimension of discriminator network")
parser.add_argument("--fea_size", type=int, default=16, help="size of feature dimension")
parser.add_argument("--latent_dim", type=int, default=6, help="dimensionality of the latent space")

opt = parser.parse_args(args=[])
print(opt)

Namespace(batch_size=128, dis_struct=[3, 1], fea_size=16, latent_dim=6, lr=0.002, max_iters=500, struct=[12, 9])


In [3]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        struct=[12,8]
        struct = opt.struct
        self.encoder = nn.Sequential(nn.Linear(int(opt.fea_size), struct[0]),
                                     nn.LeakyReLU(0.2, inplace=True),
                                     nn.Linear(struct[0], struct[1]),
                                     nn.LeakyReLU(0.2, inplace=True),
                                     nn.Linear(struct[1], int(opt.latent_dim)),
                                     nn.LeakyReLU(0.2, inplace=True))
        self.decoder = nn.Sequential(nn.Linear(int(opt.latent_dim), struct[1]),
                                     nn.LeakyReLU(0.2, inplace=True),
                                     nn.Linear(struct[1], struct[0]),
                                     nn.LeakyReLU(0.2, inplace=True),
                                     nn.Linear(struct[0], int(opt.fea_size)),
                                     nn.LeakyReLU(0.2, inplace=True))
    def forward(self, x):
        encode = self.encoder(x)
        decode = self.decoder(encode)
        return encode, decode
    
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        struct = opt.dis_struct
        self.model = nn.Sequential(
            nn.Linear(int(opt.latent_dim), struct[0]),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(struct[0], struct[1]),
            nn.Sigmoid()
        )

    def forward(self, x):
        validity = self.model(x)
        return validity


In [4]:
from __future__ import print_function
import numpy as np
import torch

import time
import random
import math
import networkx as nx
# import node2vec #随机游走相关
from utils import *
import os
# from config2 import *
# from evaluation import *
from tkinter import _flatten
import classify
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import scipy.io as sio
from sklearn.metrics import *
from sklearn.metrics.pairwise import cosine_similarity
import scipy.io as sio
from sklearn.cluster import KMeans
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn import metrics
from munkres import Munkres, print_matrix
from collections import Counter

In [5]:
# 建立边的数组
def create_A_L(inputFileName):
    f = open(inputFileName, 'r')
    lines = f.readlines()
    f.close()
    Adj_mat = []
    for line in lines[0:]:
        l = line.strip('\n\r').split(' ')
        Adj_mat.append(l[0:])
    Adj_mat = np.array(Adj_mat, dtype=np.float32)
    for i in range(157):
        for j in range(i-1,157):
            if i == j:
                Adj_mat[i][j] = 0
            else :
                Adj_mat[i][j] = Adj_mat[i][j]+Adj_mat[j][i]
                Adj_mat[j][i] = Adj_mat[i][j]
    return Adj_mat

#PPMI矩阵
def four_ord_ppmi(matrix,node_size):
    def recip_mat(matrix):
        return np.reciprocal(np.sqrt(np.sum(matrix, axis=1)))
    
    dig_value = recip_mat(matrix).tolist()
    dig_value = list(_flatten(dig_value))
    trans_mat = np.mat(np.diag(dig_value))
    trans_A = trans_mat*matrix*trans_mat
    four_ord_mat = (trans_A + trans_A*trans_A + trans_A*trans_A*trans_A + trans_A*trans_A*trans_A*trans_A )/4
    four_ord_mat_value = recip_mat(four_ord_mat)
    dig_value2 = recip_mat(four_ord_mat).tolist()
    dig_value2 = list(_flatten(dig_value2))
    trans_mat2 = np.mat(np.diag(dig_value2))
    trans_B = trans_mat2*four_ord_mat*trans_mat2
    ppmi_mat = np.log(trans_B) - np.log(1/node_size)
    ppmi_mat[np.isnan(ppmi_mat)] = 0.0
    ppmi_mat[np.isinf(ppmi_mat)] = 0.0
    ppmi_mat[np.isneginf(ppmi_mat)] = 0.0
    ppmi_mat[ppmi_mat<0] = 0.0
    return ppmi_mat

#PPMI，特征矩阵
def output_features_mat(trans_mat, feature_mat):
    node_num = feature_mat.shape[0]
    feature_num = feature_mat.shape[1]
    output_feature_mat = np.zeros_like(feature_mat)
    for i in range(node_num):
        trans_weight_sum = 0
        avg_features = np.zeros(shape=(feature_num))
        # node_i i点的与各区域连接关系
        # node_i_index_array 不为零的位置 np.nonzero()：返回数组中不为0的元素的下标
        node_i = np.array(trans_mat[i])[0]
        node_i_index_array = np.nonzero(node_i)[0]
        node_i_index_len = len(node_i_index_array)
        for j in range(node_i_index_len):
            trans_index = node_i_index_array[j]
            trans_weight = node_i[trans_index]
            avg_features += trans_weight * feature_mat[trans_index]
            trans_weight_sum += trans_weight
        avg_features /= trans_weight_sum
        output_feature_mat[i] = avg_features
    return output_feature_mat

def read_feature2(inputFileName):
    f = open(inputFileName, 'r')
    lines = f.readlines()
    f.close()
    features = []
    for line in lines[0:]:
        l = line.strip('\n\r').split(' ')
        features.append(l[0:])
    features = np.array(features, dtype=np.float32)
    return features

def read_label2(inputLabelFile):
    f = open(inputLabelFile, 'r')
    lines = f.readlines()
    f.close()
    labels = []
    for line in lines:
        l = line.strip('\n\r').split(' ')
        labels.extend(l[1:]) #将L拆分成list 删除第一个字符加入labels
    return labels


In [6]:
input_feature_mat=read_feature2('E:\Program Files\jupyter_code\code\out\out2.txt')*10000
print(input_feature_mat)

[[6.05737991e+01 2.41136990e+01 1.43002996e+01 ... 6.72195969e+01
  4.80415039e+01 2.68676987e+01]
 [1.42530000e+00 5.42000011e-02 3.73400003e-01 ... 0.00000000e+00
  5.33789968e+00 8.39600027e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [7.83899975e+00 4.55690002e+00 1.75489998e+00 ... 1.32402000e+01
  1.06758995e+01 3.77830005e+00]
 [2.85049987e+00 1.89870000e+00 1.12010002e+00 ... 4.07390022e+00
  4.00349998e+00 1.25940001e+00]
 [0.00000000e+00 2.71000005e-02 1.86700001e-01 ... 0.00000000e+00
  0.00000000e+00 4.19800013e-01]]


In [7]:
c=create_A_L('E:\Program Files\jupyter_code\code\out\out3.txt')
print(c)
cc=np.loadtxt("region_distance.csv")
print(cc)
c4 = cc-c/100
input_mat = four_ord_ppmi(c4,157)
output_feature_mat = output_features_mat(input_mat, input_feature_mat)
print(output_feature_mat)

[[0. 0. 0. ... 1. 7. 6.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 8. 0.]
 [7. 0. 0. ... 8. 0. 0.]
 [6. 0. 0. ... 0. 0. 0.]]
[[   0.  706.  676. ...  782.  798.  858.]
 [ 706.    0.  103. ... 1479. 1486. 1554.]
 [ 676.  103.    0. ... 1434. 1437. 1508.]
 ...
 [ 782. 1479. 1434. ...    0.   72.   76.]
 [ 798. 1486. 1437. ...   72.    0.   88.]
 [ 858. 1554. 1508. ...   76.   88.    0.]]
[[ 8.142838   3.1729007  2.1184618 ...  7.915345   8.012831   5.169905 ]
 [31.392109  11.102297   6.411701  ... 36.640297  30.615774  15.893806 ]
 [29.431448  10.391831   6.0624986 ... 34.127354  28.531801  15.066154 ]
 ...
 [11.188773   4.204526   2.4295828 ... 12.688712  11.199313   6.1075735]
 [10.971568   4.129916   2.3974426 ... 12.355173  10.979773   6.03045  ]
 [11.973904   4.4748163  2.5836658 ... 13.58627   11.984501   6.4776883]]


In [9]:
output_feature_mat = np.loadtxt(r"参考/output_feature.txt")
print(output_feature_mat)

[[8.41549530e+01 2.91609955e+01 1.56001406e+01 ... 1.00548904e+02
  8.41941223e+01 3.83139305e+01]
 [1.42529988e+00 5.42000011e-02 3.73399973e-01 ... 0.00000000e+00
  5.33789968e+00 8.39600027e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [9.85940247e+01 3.17165241e+01 1.79913902e+01 ... 1.06985565e+02
  9.82420197e+01 4.44198494e+01]
 [1.75184235e+02 5.79401131e+01 3.11351566e+01 ... 2.00294952e+02
  1.77323425e+02 7.88019180e+01]
 [1.02511909e+02 3.22511063e+01 1.83086128e+01 ... 1.04642761e+02
  1.01694740e+02 4.64346962e+01]]


In [10]:
ee = main(input_feature_mat, output_feature_mat)
print(ee)

reading features...
iterations: 1, time elapsed: 0.12,  autoencoder_loss:3580.93310546875
discriminator_loss:60.81591796875, generator_loss:0.08580522239208221
loss: 3641.83, iterations: 2, time elapsed: 0.12,  autoencoder_loss:4060.981689453125
discriminator_loss:60.7779426574707, generator_loss:0.08574135601520538
loss: 4121.85, iterations: 3, time elapsed: 0.13,  autoencoder_loss:3838.865966796875
discriminator_loss:60.73463821411133, generator_loss:0.08618023246526718
loss: 3899.69, iterations: 4, time elapsed: 0.14,  autoencoder_loss:3607.919189453125
discriminator_loss:60.689334869384766, generator_loss:0.08675521612167358
loss: 3668.70, iterations: 5, time elapsed: 0.15,  autoencoder_loss:3519.176513671875
discriminator_loss:60.6447639465332, generator_loss:0.08718091249465942
loss: 3579.91, iterations: 6, time elapsed: 0.15,  autoencoder_loss:3489.822509765625
discriminator_loss:60.60208511352539, generator_loss:0.08738131076097488
loss: 3550.51, iterations: 7, time elapsed: 0.

discriminator_loss:57.87937927246094, generator_loss:0.14158737659454346
loss: 3595.11, iterations: 58, time elapsed: 0.52,  autoencoder_loss:3546.981201171875
discriminator_loss:57.819644927978516, generator_loss:0.14152634143829346
loss: 3604.94, iterations: 59, time elapsed: 0.53,  autoencoder_loss:3533.66259765625
discriminator_loss:57.76066589355469, generator_loss:0.14134712517261505
loss: 3591.56, iterations: 60, time elapsed: 0.53,  autoencoder_loss:3525.412841796875
discriminator_loss:57.700321197509766, generator_loss:0.1412976086139679
loss: 3583.25, iterations: 61, time elapsed: 0.54,  autoencoder_loss:3505.669677734375
discriminator_loss:57.63996124267578, generator_loss:0.14113211631774902
loss: 3563.45, iterations: 62, time elapsed: 0.55,  autoencoder_loss:3507.304443359375
discriminator_loss:57.57915115356445, generator_loss:0.14109236001968384
loss: 3565.02, iterations: 63, time elapsed: 0.55,  autoencoder_loss:3490.07177734375
discriminator_loss:57.5189323425293, gene

loss: 3248.61, iterations: 126, time elapsed: 0.89,  autoencoder_loss:3189.311767578125
discriminator_loss:53.03932571411133, generator_loss:0.16631543636322021
loss: 3242.52, iterations: 127, time elapsed: 0.90,  autoencoder_loss:3179.88623046875
discriminator_loss:52.95936584472656, generator_loss:0.16682298481464386
loss: 3233.01, iterations: 128, time elapsed: 0.90,  autoencoder_loss:3174.677001953125
discriminator_loss:52.87825393676758, generator_loss:0.16731610894203186
loss: 3227.72, iterations: 129, time elapsed: 0.91,  autoencoder_loss:3173.5263671875
discriminator_loss:52.796104431152344, generator_loss:0.16807593405246735
loss: 3226.49, iterations: 130, time elapsed: 0.92,  autoencoder_loss:3172.80419921875
discriminator_loss:52.71376419067383, generator_loss:0.16890138387680054
loss: 3225.69, iterations: 131, time elapsed: 0.93,  autoencoder_loss:3171.359130859375
discriminator_loss:52.62987518310547, generator_loss:0.16977505385875702
loss: 3224.16, iterations: 132, time 

loss: 2769.28, iterations: 199, time elapsed: 1.45,  autoencoder_loss:2714.678466796875
discriminator_loss:46.47990036010742, generator_loss:0.2539200782775879
loss: 2761.41, iterations: 200, time elapsed: 1.46,  autoencoder_loss:2709.41650390625
discriminator_loss:46.37922286987305, generator_loss:0.25570061802864075
loss: 2756.05, iterations: 201, time elapsed: 1.47,  autoencoder_loss:2703.69287109375
discriminator_loss:46.28238296508789, generator_loss:0.2575586140155792
loss: 2750.23, iterations: 202, time elapsed: 1.47,  autoencoder_loss:2697.88037109375
discriminator_loss:46.182003021240234, generator_loss:0.25933465361595154
loss: 2744.32, iterations: 203, time elapsed: 1.48,  autoencoder_loss:2693.715087890625
discriminator_loss:46.08527755737305, generator_loss:0.2611282169818878
loss: 2740.06, iterations: 204, time elapsed: 1.49,  autoencoder_loss:2686.725830078125
discriminator_loss:45.98455047607422, generator_loss:0.26271888613700867
loss: 2732.97, iterations: 205, time el

loss: 2342.45, iterations: 275, time elapsed: 2.01,  autoencoder_loss:2298.075439453125
discriminator_loss:39.1291389465332, generator_loss:0.3576686978340149
loss: 2337.56, iterations: 276, time elapsed: 2.02,  autoencoder_loss:2295.39599609375
discriminator_loss:39.04075622558594, generator_loss:0.3586312234401703
loss: 2334.80, iterations: 277, time elapsed: 2.03,  autoencoder_loss:2290.823486328125
discriminator_loss:38.95048904418945, generator_loss:0.35955721139907837
loss: 2330.13, iterations: 278, time elapsed: 2.04,  autoencoder_loss:2286.2939453125
discriminator_loss:38.865509033203125, generator_loss:0.36048802733421326
loss: 2325.52, iterations: 279, time elapsed: 2.04,  autoencoder_loss:2282.661376953125
discriminator_loss:38.77621841430664, generator_loss:0.3613698184490204
loss: 2321.80, iterations: 280, time elapsed: 2.05,  autoencoder_loss:2278.034912109375
discriminator_loss:38.68890380859375, generator_loss:0.3621000051498413
loss: 2317.09, iterations: 281, time elap

loss: 2047.22, iterations: 353, time elapsed: 2.58,  autoencoder_loss:2010.2186279296875
discriminator_loss:33.343807220458984, generator_loss:0.420456200838089
loss: 2043.98, iterations: 354, time elapsed: 2.59,  autoencoder_loss:2007.0010986328125
discriminator_loss:33.28099822998047, generator_loss:0.421318918466568
loss: 2040.70, iterations: 355, time elapsed: 2.59,  autoencoder_loss:2004.6878662109375
discriminator_loss:33.220069885253906, generator_loss:0.42225757241249084
loss: 2038.33, iterations: 356, time elapsed: 2.60,  autoencoder_loss:2002.371826171875
discriminator_loss:33.16402053833008, generator_loss:0.42325976490974426
loss: 2035.96, iterations: 357, time elapsed: 2.61,  autoencoder_loss:1999.4390869140625
discriminator_loss:33.11247253417969, generator_loss:0.4242086410522461
loss: 2032.98, iterations: 358, time elapsed: 2.62,  autoencoder_loss:1997.137451171875
discriminator_loss:33.05558776855469, generator_loss:0.42513442039489746
loss: 2030.62, iterations: 359, t

loss: 1900.44, iterations: 405, time elapsed: 2.95,  autoencoder_loss:1865.9390869140625
discriminator_loss:30.78023338317871, generator_loss:0.46893879771232605
loss: 1897.19, iterations: 406, time elapsed: 2.96,  autoencoder_loss:1863.892822265625
discriminator_loss:30.742584228515625, generator_loss:0.4699341356754303
loss: 1895.11, iterations: 407, time elapsed: 2.97,  autoencoder_loss:1861.882568359375
discriminator_loss:30.703170776367188, generator_loss:0.47105640172958374
loss: 1893.06, iterations: 408, time elapsed: 2.97,  autoencoder_loss:1860.045166015625
discriminator_loss:30.660844802856445, generator_loss:0.4721556305885315
loss: 1891.18, iterations: 409, time elapsed: 2.98,  autoencoder_loss:1857.501220703125
discriminator_loss:30.62216567993164, generator_loss:0.47313663363456726
loss: 1888.60, iterations: 410, time elapsed: 2.99,  autoencoder_loss:1854.9759521484375
discriminator_loss:30.57979393005371, generator_loss:0.4741106331348419
loss: 1886.03, iterations: 411, 

loss: 1740.35, iterations: 479, time elapsed: 3.52,  autoencoder_loss:1710.0487060546875
discriminator_loss:28.215261459350586, generator_loss:0.5480449795722961
loss: 1738.81, iterations: 480, time elapsed: 3.53,  autoencoder_loss:1708.0369873046875
discriminator_loss:28.18540382385254, generator_loss:0.5492469072341919
loss: 1736.77, iterations: 481, time elapsed: 3.54,  autoencoder_loss:1705.5869140625
discriminator_loss:28.155651092529297, generator_loss:0.5502766370773315
loss: 1734.29, iterations: 482, time elapsed: 3.54,  autoencoder_loss:1704.4459228515625
discriminator_loss:28.125463485717773, generator_loss:0.5514798164367676
loss: 1733.12, iterations: 483, time elapsed: 3.55,  autoencoder_loss:1702.45458984375
discriminator_loss:28.09759521484375, generator_loss:0.5527277588844299
loss: 1731.10, iterations: 484, time elapsed: 3.56,  autoencoder_loss:1700.9232177734375
discriminator_loss:28.067909240722656, generator_loss:0.5540988445281982
loss: 1729.55, iterations: 485, tim

          2.6835e+00]], grad_fn=<LeakyReluBackward1>)
tensor([[-3.8174e+00,  1.2043e+01,  3.7589e+00, -3.6332e+00,  1.1897e+01,
          9.1726e+00],
        [-2.8365e-01,  1.1723e+00, -2.3386e-01, -1.0315e-01,  7.1960e-01,
          4.6394e-01],
        [-2.9373e-01,  1.1536e+00,  3.1426e-01, -3.0241e-01,  1.9682e+00,
          1.7382e+00],
        [-2.9373e-01,  1.1536e+00,  3.1426e-01, -3.0241e-01,  1.9682e+00,
          1.7382e+00],
        [-3.1422e-01,  1.2359e+00,  3.9599e-01, -3.3477e-01,  2.1591e+00,
          1.9293e+00],
        [-6.4993e-01,  2.5793e+00,  1.7745e+00, -7.4083e-01,  4.2161e+00,
          4.3541e+00],
        [-2.8618e+00,  9.8631e+00,  1.0545e+01, -3.5423e+00,  1.8692e+01,
          2.1098e+01],
        [-1.9456e+00,  7.6953e+00,  6.8817e+00, -2.5213e+00,  1.2699e+01,
          1.4129e+01],
        [-2.9373e-01,  1.1536e+00,  3.1426e-01, -3.0241e-01,  1.9682e+00,
          1.7382e+00],
        [-4.8038e-01,  1.6385e+00,  9.6246e-01, -4.9008e-01,  2.8227e+00,

In [26]:
ee = ee.detach().numpy()
np.savetxt('fea3.txt',ee)


In [8]:
def main(input_feature_mat, output_feature_mat):
    #读文件
    # Read features
    print('reading features...')
    X = input_feature_mat
    N = X.shape[0]
    dims = opt.latent_dim
    X_target = output_feature_mat
    
#     X=torch.from_numpy(X)
#     X_target = torch.from_numpy(X_target)
    
    
    # Initialize generator and discriminator
    generator = Generator()
    discriminator = Discriminator()

    # ----------
    #  Training
    # ----------
    batch_size = opt.batch_size
    max_iters = opt.max_iters

    idx = 0
    print_every_k_iterations = 1
    start = time.time()

    loss_sg = 0
    loss_ae = 0
    loss_dis = 0
    loss_gene = 0
    
    # Loss function
    adversarial_loss = torch.nn.BCELoss()
    criterion = torch.nn.MSELoss()
    
    
    # Optimizers
    optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr)
    optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, weight_decay=0.01)
    optimizer_G2 = torch.optim.Adam(generator.encoder.parameters(), lr=opt.lr)

    cuda = False
    Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

    
    
    for epoch in range(max_iters):
        idx += 1


        # train for autoencoder model
        start_idx = np.random.randint(0, N - batch_size)
        batch_idx = np.array(range(start_idx, start_idx + batch_size))
        batch_idx = np.random.permutation(batch_idx)
        batch_X = X[batch_idx]
        X_new = X_target[batch_idx]
        X_new = Variable(Tensor(X_new))
        
        batch_X = Variable(Tensor(batch_X))
        batch_Y, batch_XX = generator(batch_X)
        loss_ae_value = criterion(X_new, batch_XX)
        optimizer_G.zero_grad()
        
        loss_ae_value.backward(retain_graph=True)
        
        optimizer_G.step()
        
        loss_ae += loss_ae_value
        
        #train for discriminator 
        z_real_dist = Variable(torch.Tensor(batch_size, dims).uniform_(0,1))
        d_g_real = discriminator(z_real_dist)
        d_g_fake = discriminator(batch_Y.detach())
        
        alpha_g = Variable(Tensor(batch_size, dims).uniform_(0,1))
        interpolates_g = alpha_g*d_g_real + ((1-alpha_g)*d_g_fake)
        disc_g_interpolates = discriminator(interpolates_g)
        z=torch.ones(128,1)
        
        gradients_g = torch.autograd.grad(disc_g_interpolates, [interpolates_g], grad_outputs=z, create_graph=True)[0]
        slopes_g = gradients_g.pow(2).sum(1).sqrt()
        gradient_penalty_g = (slopes_g-1).pow(2).sum()
        
        # Adversarial ground truths        
        valid = Variable(Tensor(d_g_real.size()).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(d_g_fake.size()).fill_(0.0), requires_grad=False)
        
        dc_g_loss_real = adversarial_loss(d_g_real, valid)
        dc_g_loss_fake = adversarial_loss(d_g_fake, fake)
        dc_g_loss = 0.5*(dc_g_loss_fake + dc_g_loss_real + gradient_penalty_g)
        loss_dis_value = dc_g_loss
        optimizer_D.zero_grad()
        loss_dis_value.backward(retain_graph=True)
        optimizer_D.step()
        
        loss_dis += loss_dis_value
        
        #train for generator
        optimizer_G2.zero_grad()
        generator_g_loss = 0.2*adversarial_loss(d_g_fake,torch.ones_like(d_g_fake))
        loss_gene_value = generator_g_loss
        loss_gene_value.backward(retain_graph=True)
        optimizer_G2.step()
        loss_gene += loss_gene_value
        
        
        if idx % print_every_k_iterations == 0:
            end = time.time()
            print('iterations: %d' % idx + ', time elapsed: %.2f, ' % (end - start), end='')
            print(" autoencoder_loss:{}".format(loss_ae/idx))
            print("discriminator_loss:{}, generator_loss:{}".format(loss_dis/idx, loss_gene/idx))
            total_loss = loss_sg/idx + loss_ae/idx + loss_dis/idx + loss_gene/idx
            print('loss: %.2f, ' % total_loss, end='')


    print('optimization finished...')
    X=torch.from_numpy(X)
    generator.eval()
    embedding_result ,_ = generator(X)
    print(embedding_result)
    return embedding_result